In [2]:
import requests_cache
import numpy as np
import pandas as pd
import os
import re
# from dotenv import load_dotenv
from sqlalchemy import create_engine
from modules.utils import *

In [5]:
# Create the SQL engine
engine = create_engine(f"mysql+mariadbconnector://{os.environ['user']}:{os.environ['password']}@{os.environ['host']}:3306/{os.environ['database']}")

# Initialize logger and cache session
logger = add_custom_logger('etl', file_path='logs/etl.log', level=logging.INFO, write_mode='w')
session = requests_cache.CachedSession('logs/address_cache', backend="sqlite")

logger.info(f"{'-'*50}New run started {'-'*50}")

## Get unique addresses from Staging table

In [32]:
query = '''
        SELECT *
        FROM stg_addresses
        '''
df = pd.read_sql(query, con=engine)
df

,town,street_name,block_number,full_address
0,ANG MO KIO,ANG MO KIO AVE 1,205,"205, ANG MO KIO AVE 1"
1,ANG MO KIO,ANG MO KIO AVE 1,207,"207, ANG MO KIO AVE 1"
2,ANG MO KIO,ANG MO KIO AVE 1,208,"208, ANG MO KIO AVE 1"
3,ANG MO KIO,ANG MO KIO AVE 1,215,"215, ANG MO KIO AVE 1"
4,ANG MO KIO,ANG MO KIO AVE 1,216,"216, ANG MO KIO AVE 1"
...,...,...,...,...
9518,YISHUN,YISHUN ST 81,876,"876, YISHUN ST 81"
9519,YISHUN,YISHUN ST 81,877,"877, YISHUN ST 81"
9520,YISHUN,YISHUN ST 81,878,"878, YISHUN ST 81"
9521,YISHUN,YISHUN ST 81,879,"879, YISHUN ST 81"


In [33]:
# Logger here is the debugger logger
logger.info(f"{'-'*50}New run started {'-'*50}")

# Getting latitude, longitude, postal code
def get_lat_long(address_df : pd.DataFrame, verbose: int=1):
    '''
    The actual API call to be called row-wise to get latitude, longitude, and postal code
    ## Parameters
    address_df : pd.DataFrame
        DataFrame that contains a combination of ['block_number'] and ['street_name'] as ['full_address']
    '''

    # API call
    try:
        address = address_df['full_address']
        call = f"https://www.onemap.gov.sg/api/common/elastic/search?searchVal={address}&returnGeom=Y&getAddrDetails=Y"
        # Caching is enabled in the session
        response = session.get(call)
        response.raise_for_status()
        data = response.json()
        if verbose ==1:
            logger.info(f'Response: {response.status_code} \tGet request call: {response.url}')

        # For those with ST abbreviations
        if len(data['results'])<1:
            address = re.sub(' ST', ' STREET', address_df['full_address'])
            call = f"https://www.onemap.gov.sg/api/common/elastic/search?searchVal={address}&returnGeom=Y&getAddrDetails=Y"
            # Caching is enabled in the session
            response = session.get(call)
            response.raise_for_status()
            data = response.json()
            if verbose ==1:
                logger.info(f'Response: {response.status_code} \tGet request call: {response.url}')

        return [float(data['results'][0]['LATITUDE']), float(data['results'][0]['LONGITUDE']), data['results'][0]['POSTAL']]

    except Exception as err:
        logger.error(f'Error occurred - get_lat_long() API call: {err} on the following call:', exc_info=True)
        return [np.NaN, np.NaN, np.NaN ]

# This calls the API call function row wise
df[['latitude', 'longitude', 'postal_code']] = df.apply(get_lat_long, axis=1, result_type='expand')

## Full refresh onto DB

In [38]:
df.to_sql("geolocations", engine, if_exists="replace", index=False) 

9523

In [35]:
query = '''
        SELECT *
        FROM geolocations
        '''
df = pd.read_sql(query, con=engine)
df

,town,street_name,block_number,full_address,latitude,longitude,postal_code
0,ANG MO KIO,ANG MO KIO AVE 1,205,"205, ANG MO KIO AVE 1",1.366941,103.843582,560205
1,ANG MO KIO,ANG MO KIO AVE 1,207,"207, ANG MO KIO AVE 1",1.365821,103.842848,560207
2,ANG MO KIO,ANG MO KIO AVE 1,208,"208, ANG MO KIO AVE 1",1.365445,103.842715,560208
3,ANG MO KIO,ANG MO KIO AVE 1,215,"215, ANG MO KIO AVE 1",1.366558,103.841624,560215
4,ANG MO KIO,ANG MO KIO AVE 1,216,"216, ANG MO KIO AVE 1",1.366197,103.841505,560216
...,...,...,...,...,...,...,...
9518,YISHUN,YISHUN ST 81,876,"876, YISHUN ST 81",1.414745,103.835532,760876
9519,YISHUN,YISHUN ST 81,877,"877, YISHUN ST 81",1.413902,103.835454,760877
9520,YISHUN,YISHUN ST 81,878,"878, YISHUN ST 81",1.414053,103.835888,760878
9521,YISHUN,YISHUN ST 81,879,"879, YISHUN ST 81",1.414442,103.836118,760879


## Manual Patching

In [36]:
df[df['postal_code']=='NIL']

,town,street_name,block_number,full_address,latitude,longitude,postal_code
2503,CHOA CHU KANG,CHOA CHU KANG CTRL,215,"215, CHOA CHU KANG CTRL",1.383083,103.747077,NIL
2504,CHOA CHU KANG,CHOA CHU KANG CTRL,216,"216, CHOA CHU KANG CTRL",1.383083,103.747077,NIL
2725,CHOA CHU KANG,TECK WHYE AVE,4,"4, TECK WHYE AVE",1.378671,103.756148,NIL
5436,PASIR RIS,PASIR RIS ST 71,772,"772, PASIR RIS ST 71",1.377463,103.934760,NIL
5437,PASIR RIS,PASIR RIS ST 71,773,"773, PASIR RIS ST 71",1.374198,103.938645,NIL


In [37]:
df.loc[df['full_address'] == '215, CHOA CHU KANG CTRL', 'postal_code'] = '680215'
df.loc[df['full_address'] == '216, CHOA CHU KANG CTRL', 'postal_code'] = '680216'
df.loc[df['full_address'] == '4, TECK WHYE AVE', 'postal_code'] = '680004'
df.loc[df['full_address'] == '772, PASIR RIS ST 71', 'postal_code'] = '510772'
df.loc[df['full_address'] == '773, PASIR RIS ST 71', 'postal_code'] = '510773'

In [41]:
df.to_csv('ingestion/geodata.csv', index=False)